In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import folium
import requests 
from geopy.geocoders import Nominatim
import re as regex

use request to connect to wikipedia page and use beautifulsoup to process html

In [ ]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req=urllib.request.urlopen(url)
article=req.read().decode()
with open('localfile.html', 'w') as fo:
    fo.write(article)
article

article=open('localfile.html').read()
soup=BeautifulSoup(article,'html.parser')
tables=soup.find_all('table',class_='sortable')
tables

content to dataframe (at the time when I process wikipedia data, neighborhoods are already collapsed according to Borough)

In [ ]:
results_table=[]
for tr in tables:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    results_table.append(row)
results_table=pd.DataFrame(np.array(results_table[0]).reshape((-1,3)))
results_table.columns=['PostalCode','Borough','Neighborhood']
results_table=results_table[results_table['Borough']!='Not assigned'].reset_index(drop=True)
results_table

In [ ]:
results_table.shape


download geographial information

In [ ]:
!wget -q -O 'geo.csv' https://cocl.us/Geospatial_data

merge two dataset

In [ ]:
geo=pd.DataFrame.from_csv('geo.csv')
results_table=results_table.merge(geo,left_on='PostalCode',right_index=True)
results_table


create a map for toronto

In [ ]:
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(results_table['Latitude'], results_table['Longitude'], results_table['Borough'], results_table['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
toronto_data = results_table[results_table['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

four square information

In [ ]:
CLIENT_ID = '50OG1L24VQHMKEDMSCTEKVCRVM3YZ2LLOOGNCNN4QVKN2SGY' # your Foursquare ID
CLIENT_SECRET = 'QMI5S45Y5GFI0MJT0BGORIW23LEFUNNMU1FE4UZJ5V4DOHGG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
toronto_data.loc[0, 'Neighborhood']
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


In [ ]:
toronto_venues


In [ ]:
toronto_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
all_restaurants=[]
for col in toronto_onehot.columns: 
    if regex.findall("Restaurant",col):
        all_restaurants.append(col)
all_restaurants

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 208

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

In [ ]:
all_restaurants
asian_restaurants=['Asian Restaurant','Chinese Restaurant','Japanese Restaurant','Korean Restaurant','Sushi Restaurant','Taiwanese Restaurant','Thai Restaurant','Vietnamese Restaurant']

In [ ]:
toronto_grouped['asian']=toronto_grouped[asian_restaurants].sum(axis=1)
toronto_grouped['diff']=toronto_grouped['asian']-toronto_grouped['Korean Restaurant']

In [ ]:
toronto_grouped[['Neighborhood','diff']]

In [ ]:
num=np.argmax(toronto_grouped['asian']-toronto_grouped['Korean Restaurant'])

In [ ]:
toronto_grouped['Neighborhood'][num]